In [1]:
import os
os.chdir("D:\\UIT\\HK\\HK6\\IS403\\Đồ án\\Source code")
print("Thư mục làm việc hiện tại:", os.getcwd())

Thư mục làm việc hiện tại: D:\UIT\HK\HK6\IS403\Đồ án\Source code


# Import modules

In [2]:
import pandas as pd

In [3]:
import joblib

In [4]:
import json

In [5]:
import time

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

In [7]:
from model_application.evaluation_errors_utils import error

# Load and preprocess data

In [ ]:
def load_and_prepare_data(path):
    dataframe = pd.read_csv(path)
    dataframe["Time"] = pd.to_datetime(dataframe["Time"])

    dataframe["Close_lag1"] = dataframe["Close"].shift(1)
    dataframe["MA5"] = dataframe["Close"].rolling(5).mean()
    dataframe["Return_1day"] = dataframe["Close"].pct_change(1)

    dataframe = dataframe.dropna()
    X = dataframe.drop(columns=["Close", "Time"])
    y = dataframe["Close"]
    
    return X, y


def scale_data(X_train, X_test, scaler_path, bank):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    joblib.dump(scaler, os.path.join(scaler_path, f"{bank}_scaler"))
    return X_train_scaled, X_test_scaled

In [9]:
scaler_path = ".\\model_application\\experiment\\saved_checkpoint\\machine_learning\\scaler"

## Vietcombank

In [10]:
vcb_X_train, vcb_y_train = load_and_prepare_data(".\\dataset\\official_dataset\\vcb\\vcb_train.csv")
vcb_X_validation, vcb_y_validation = load_and_prepare_data(".\\dataset\\official_dataset\\vcb\\vcb_validation.csv")
vcb_X_test, vcb_y_test = load_and_prepare_data(".\\dataset\\official_dataset\\vcb\\vcb_test.csv")

In [11]:
vcb_X_train = pd.concat([vcb_X_train, vcb_X_validation])
vcb_y_train = pd.concat([vcb_y_train, vcb_y_validation])

In [12]:
vcb_X_train

,Previous_GDP,Previous_EUR_Cash,Previous_THB_Cash,Previous_USD_Cash,Tăng trưởng lợi nhuận (%),Vốn hóa (Tỷ đồng),BVPS (VND),VỐN CHỦ SỞ HỮU (đồng),Close_lag1,MA5,Return_1day
4,1467490,25013.62,595.78,22540.0,0.153907,382860590593900,23038.541622,111170655000000,45.20,44.790,-0.003761
5,1467490,25033.58,593.93,22510.0,0.153907,382860590593900,23038.541622,111170655000000,45.03,44.836,-0.001332
6,1467490,25064.52,597.29,22530.0,0.153907,382860590593900,23038.541622,111170655000000,44.97,45.062,0.007561
7,1467490,25148.56,602.50,22530.0,0.153907,382860590593900,23038.541622,111170655000000,45.31,45.378,0.023615
8,1467490,25303.38,602.68,22530.0,0.153907,382860590593900,23038.541622,111170655000000,46.38,45.784,0.018327
...,...,...,...,...,...,...,...,...,...,...,...
80,1561963,25792.47,657.38,25193.0,0.178524,512519668725400,34028.114605,190297001000000,61.61,61.782,0.003246
81,1561963,25779.62,658.40,25206.0,0.178524,512519668725400,34028.114605,190297001000000,61.81,61.742,-0.003236
82,1561963,25747.18,657.18,25195.0,0.178524,512519668725400,34028.114605,190297001000000,61.61,61.702,0.000974
83,1561963,25817.03,658.27,25208.0,0.178524,512519668725400,34028.114605,190297001000000,61.67,61.648,-0.002108


In [13]:
vcb_y_train

4     45.03
5     44.97
6     45.31
7     46.38
8     47.23
      ...  
80    61.81
81    61.61
82    61.67
83    61.54
84    61.00
Name: Close, Length: 825, dtype: float64

In [14]:
vcb_X_train_scaled, vcb_X_test_scaled = scale_data(vcb_X_train, vcb_X_test, scaler_path, "vcb")

In [15]:
vcb_X_train_scaled

array([[ 0.16858864, -0.23530154, -0.62668709, ..., -1.25108706,
        -1.30793547, -0.30289451],
       [ 0.16858864, -0.21799191, -0.69529661, ..., -1.27412569,
        -1.30168077, -0.12921474],
       [ 0.16858864, -0.19116024, -0.57068689, ..., -1.28225697,
        -1.27095116,  0.50676094],
       ...,
       [ 1.05682222,  0.40085358,  1.65040717, ...,  0.97281821,
         0.99161871,  0.0357186 ],
       [ 1.05682222,  0.46142863,  1.69083116, ...,  0.98094949,
         0.98427624, -0.18467725],
       [ 1.05682222,  0.49603923,  1.84436813, ...,  0.96333172,
         0.96768768, -0.66144488]])

## BIDV

In [16]:
bid_X_train, bid_y_train = load_and_prepare_data(".\\dataset\\official_dataset\\bid\\bid_train.csv")
bid_X_validation, bid_y_validation = load_and_prepare_data(".\\dataset\\official_dataset\\bid\\bid_validation.csv")
bid_X_test, bid_y_test = load_and_prepare_data(".\\dataset\\official_dataset\\bid\\bid_test.csv")

In [17]:
bid_X_train = pd.concat([bid_X_train, bid_X_validation])
bid_y_train = pd.concat([bid_y_train, bid_y_validation])

In [18]:
bid_X_train

,Previous_GDP,Previous_EUR_Cash,Previous_THB_Cash,Previous_USD_Cash,Tăng trưởng lợi nhuận (%),Vốn hóa (Tỷ đồng),BVPS (VND),VỐN CHỦ SỞ HỮU (đồng),Close_lag1,MA5,Return_1day
4,1467490,25209.0,636.69,22495.0,0.355021,188430012146000,16434.655845,86366645000000,28.75,28.022,0.004870
5,1467490,25270.0,640.67,22535.0,0.355021,188430012146000,16434.655845,86366645000000,28.89,28.718,0.069920
6,1467490,25328.0,643.55,22515.0,0.355021,188430012146000,16434.655845,86366645000000,30.91,29.738,0.043999
7,1467490,25481.0,646.85,22515.0,0.355021,188430012146000,16434.655845,86366645000000,32.27,30.654,0.005578
8,1467490,25573.0,647.33,22525.0,0.355021,188430012146000,16434.655845,86366645000000,32.45,31.306,-0.013559
...,...,...,...,...,...,...,...,...,...,...,...
75,1561963,25981.0,684.02,24219.0,0.124035,269003095452000,19333.031073,138439656000000,38.10,38.426,0.014436
76,1561963,26014.0,683.64,24227.0,0.124035,269003095452000,19333.031073,138439656000000,38.65,38.548,0.009056
77,1561963,25983.0,682.94,24216.0,0.124035,269003095452000,19333.031073,138439656000000,39.00,38.700,0.003846
78,1561963,26018.0,683.55,24209.0,0.124035,269003095452000,19333.031073,138439656000000,39.15,38.630,-0.022989


In [19]:
bid_y_train

4     28.89
5     30.91
6     32.27
7     32.45
8     32.01
      ...  
75    38.65
76    39.00
77    39.15
78    38.25
79    37.55
Name: Close, Length: 769, dtype: float64

In [20]:
bid_X_train_scaled, bid_X_test_scaled = scale_data(bid_X_train, bid_X_test, scaler_path, "bid")

In [21]:
bid_X_train_scaled

array([[ 0.16786435, -0.2321394 , -0.73249247, ..., -0.98181291,
        -1.11549947,  0.22156642],
       [ 0.16786435, -0.17998573, -0.55790419, ..., -0.95648549,
        -0.98909844,  3.49046651],
       [ 0.16786435, -0.13039698, -0.43156895, ..., -0.59104694,
        -0.80385556,  2.18786364],
       ...,
       [ 1.05669924,  0.42961383,  1.2963286 , ...,  0.87251635,
         0.82373922,  0.17013848],
       [ 1.05669924,  0.45953807,  1.32308711, ...,  0.89965287,
         0.81102647, -1.17834287],
       [ 1.05669924,  0.51596664,  1.562159  , ...,  0.73683372,
         0.7910493 , -0.94277136]])

## Techcombank

In [22]:
tcb_X_train, tcb_y_train = load_and_prepare_data(".\\dataset\\official_dataset\\tcb\\tcb_train.csv")
tcb_X_validation, tcb_y_validation = load_and_prepare_data(".\\dataset\\official_dataset\\tcb\\tcb_validation.csv")
tcb_X_test, tcb_y_test = load_and_prepare_data(".\\dataset\\official_dataset\\tcb\\tcb_test.csv")

In [23]:
tcb_X_train = pd.concat([tcb_X_train, tcb_X_validation])
tcb_y_train = pd.concat([tcb_y_train, tcb_y_validation])

In [24]:
tcb_X_train

,Previous_GDP,Previous_EUR_Cash,Previous_THB_Cash,Previous_USD_Cash,Tăng trưởng lợi nhuận (%),Vốn hóa (Tỷ đồng),BVPS (VND),VỐN CHỦ SỞ HỮU (đồng),Close_lag1,MA5,Return_1day
4,1467490,25188.0,596.0,22521.0,0.161678,160097714788800,26259.416507,93055837000000,24.44,24.178,0.002046
5,1467490,25432.0,599.0,22535.0,0.161678,160097714788800,26259.416507,93055837000000,24.49,24.104,-0.015108
6,1467490,25371.0,604.0,22557.0,0.161678,160097714788800,26259.416507,93055837000000,24.12,24.036,-0.019900
7,1467490,25308.0,605.0,22574.0,0.161678,160097714788800,26259.416507,93055837000000,23.64,24.076,0.002115
8,1467490,25122.0,601.0,22525.0,0.161678,160097714788800,26259.416507,93055837000000,23.69,23.926,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
62,1561963,25864.0,660.0,25225.0,0.243784,166730501040400,20259.284053,144367982000000,23.80,23.990,0.008403
63,1561963,25810.0,660.0,25210.0,0.243784,166730501040400,20259.284053,144367982000000,24.00,24.040,0.018750
64,1561963,25848.0,659.0,25205.0,0.243784,166730501040400,20259.284053,144367982000000,24.45,24.070,-0.004090
65,1561963,25828.0,658.0,25210.0,0.243784,166730501040400,20259.284053,144367982000000,24.35,24.240,0.010267


In [25]:
tcb_y_train

4     24.49
5     24.12
6     23.64
7     23.69
8     23.69
      ...  
62    24.00
63    24.45
64    24.35
65    24.60
66    24.65
Name: Close, Length: 651, dtype: float64

In [26]:
tcb_X_train_scaled, tcb_X_test_scaled = scale_data(tcb_X_train, tcb_X_test, scaler_path, "tcb")

In [27]:
tcb_X_train_scaled

array([[ 0.17615086, -0.13154369, -0.44261427, ...,  1.26377878,
         1.20936925,  0.07423094],
       [ 0.17615086,  0.07534291, -0.34334628, ...,  1.27531963,
         1.19221591, -0.70982447],
       [ 0.17615086,  0.02362126, -0.17789963, ...,  1.18991729,
         1.17645338, -0.92886455],
       ...,
       [ 1.05991596,  0.42806762,  1.64201354, ...,  1.26608695,
         1.18433465, -0.20621691],
       [ 1.05991596,  0.4111097 ,  1.60892421, ...,  1.24300523,
         1.22374097,  0.44999147],
       [ 1.05991596,  0.52981513,  1.77437086, ...,  1.30070952,
         1.26314729,  0.07362276]])

# Excute model

In [27]:
models = {
    "random_forest": {
        "model": RandomForestRegressor(random_state=42),
        "params": {
            "n_estimators": [100, 200],
            "max_depth": [None, 10]
        },
        "use_scaled": False,
        "weight": 0.8
    },
    "svm": {
        "model": SVR(),
        "params": {
            "kernel": ["linear", "rbf"],
            "C": [0.1, 1, 10],
            "epsilon": [0.01, 0.1]
        },
        "use_scaled": True,
        "weight": 0.7
    },
    "linear_regression": {
        "model": LinearRegression(),
        "params": {},
        "use_scaled": True,
        "weight": 0.6
    },
    "ridge": {
        "model": Ridge(),
        "params": {
            "alpha": [0.1, 1.0, 10.0]
        },
        "use_scaled": True,
        "weight": 0.5
    }
}

In [28]:
def find_best_models(X_train, X_train_scaled, y_train, checkpoint_path):
    best_models = {}
    for name, config in models.items():
        start_time = time.time()
        print(f"\nĐang tối ưu mô hình {name}...")
        X_tr = X_train_scaled if config["use_scaled"] else X_train

        if config["params"]:
            tscv = TimeSeriesSplit(n_splits=5)
            gscv = GridSearchCV(
                estimator=config["model"],
                param_grid=config["params"],
                cv=tscv,
                scoring="neg_mean_squared_error",
                n_jobs=-1,
                verbose=0
            )
            gscv.fit(X_tr, y_train)
            best_models[name] = gscv.best_estimator_
            print(f"Best params: {gscv.best_params_}")
        else:
            model = config["model"]
            model.fit(X_tr, y_train)
            best_models[name] = gscv.best_estimator_
            print(f"Best params: {gscv.best_params_}")
        end_time = time.time()

        training_time = end_time - start_time
        json.dump(
            {"training time": training_time},
            open(os.path.join(checkpoint_path, f"{name}_training_time.json"), "w+"),
            ensure_ascii=False,
            indent=4
        )

    for name, model in best_models.items():
        file_path = os.path.join(checkpoint_path, f"{name}.pkl")
        with open(file_path, "wb") as file:
            joblib.dump(model, file)
        print(f"Đã lưu mô hình {name} vào file: {file_path}")
    

def evaluate(X_test, X_test_scaled, y_test, checkpoint_path):
    for name, config in models.items():
        X_t = X_test_scaled if config["use_scaled"] else X_test
        model = joblib.load(os.path.join(checkpoint_path, f"{name}.pkl"))

        start_time = time.time()
        predictions = model.predict(X_t)
        end_time = time.time()

        mse, rmse, mae, mape = error(predictions, y_test)
        elapsed = end_time - start_time

        json.dump(
            {
                "time": elapsed,
                "predictions": predictions.tolist(),
                "targets": y_test.values.tolist(),
                "mse": mse,
                "rmse": rmse,
                "mae": mae,
                "mape": mape
            },
            open(os.path.join(checkpoint_path, f"{name}_test_results.json"), "w+"),
            ensure_ascii=False,
            indent=4
        )


def get_predictions(features, checkpoint_path, model_name):
    model = joblib.load(os.path.join(checkpoint_path, f"{model_name}.pkl"))
    predictions = model.predict(features)

    json.dump(
        {"predictions": predictions.tolist()},
        open(os.path.join(checkpoint_path, "predictions.json"), "w+"),
        ensure_ascii=False,
        indent=4
    )

## Vietcombank

In [29]:
vcb_checkpoint_path = ".\\model_application\\experiment\\saved_checkpoint\\machine_learning\\one_step\\vcb"

In [30]:
find_best_models(vcb_X_train, vcb_X_train_scaled, vcb_y_train, vcb_checkpoint_path)


Đang tối ưu mô hình random_forest...
Best params: {'max_depth': 10, 'n_estimators': 200}

Đang tối ưu mô hình svm...
Best params: {'C': 10, 'epsilon': 0.01, 'kernel': 'linear'}

Đang tối ưu mô hình linear_regression...
Best params: {'C': 10, 'epsilon': 0.01, 'kernel': 'linear'}

Đang tối ưu mô hình ridge...
Best params: {'alpha': 0.1}
Đã lưu mô hình random_forest vào file: .\model_application\experiment\saved_checkpoint\machine_learning\one_step\vcb\random_forest.pkl
Đã lưu mô hình svm vào file: .\model_application\experiment\saved_checkpoint\machine_learning\one_step\vcb\svm.pkl
Đã lưu mô hình linear_regression vào file: .\model_application\experiment\saved_checkpoint\machine_learning\one_step\vcb\linear_regression.pkl
Đã lưu mô hình ridge vào file: .\model_application\experiment\saved_checkpoint\machine_learning\one_step\vcb\ridge.pkl


In [31]:
evaluate(vcb_X_test, vcb_X_test_scaled, vcb_y_test, vcb_checkpoint_path)

## BIDV

In [32]:
bid_checkpoint_path = ".\\model_application\\experiment\\saved_checkpoint\\machine_learning\\one_step\\bid"

In [33]:
find_best_models(bid_X_train, bid_X_train_scaled, bid_y_train, bid_checkpoint_path)


Đang tối ưu mô hình random_forest...
Best params: {'max_depth': None, 'n_estimators': 200}

Đang tối ưu mô hình svm...
Best params: {'C': 10, 'epsilon': 0.01, 'kernel': 'linear'}

Đang tối ưu mô hình linear_regression...
Best params: {'C': 10, 'epsilon': 0.01, 'kernel': 'linear'}

Đang tối ưu mô hình ridge...
Best params: {'alpha': 0.1}
Đã lưu mô hình random_forest vào file: .\model_application\experiment\saved_checkpoint\machine_learning\one_step\bid\random_forest.pkl
Đã lưu mô hình svm vào file: .\model_application\experiment\saved_checkpoint\machine_learning\one_step\bid\svm.pkl
Đã lưu mô hình linear_regression vào file: .\model_application\experiment\saved_checkpoint\machine_learning\one_step\bid\linear_regression.pkl
Đã lưu mô hình ridge vào file: .\model_application\experiment\saved_checkpoint\machine_learning\one_step\bid\ridge.pkl


In [34]:
evaluate(bid_X_test, bid_X_test_scaled, bid_y_test, bid_checkpoint_path)

## Techcombank

In [35]:
tcb_checkpoint_path = ".\\model_application\\experiment\\saved_checkpoint\\machine_learning\\one_step\\tcb"

In [36]:
find_best_models(tcb_X_train, tcb_X_train_scaled, tcb_y_train, tcb_checkpoint_path)


Đang tối ưu mô hình random_forest...
Best params: {'max_depth': None, 'n_estimators': 100}

Đang tối ưu mô hình svm...
Best params: {'C': 10, 'epsilon': 0.1, 'kernel': 'linear'}

Đang tối ưu mô hình linear_regression...
Best params: {'C': 10, 'epsilon': 0.1, 'kernel': 'linear'}

Đang tối ưu mô hình ridge...
Best params: {'alpha': 0.1}
Đã lưu mô hình random_forest vào file: .\model_application\experiment\saved_checkpoint\machine_learning\one_step\tcb\random_forest.pkl
Đã lưu mô hình svm vào file: .\model_application\experiment\saved_checkpoint\machine_learning\one_step\tcb\svm.pkl
Đã lưu mô hình linear_regression vào file: .\model_application\experiment\saved_checkpoint\machine_learning\one_step\tcb\linear_regression.pkl
Đã lưu mô hình ridge vào file: .\model_application\experiment\saved_checkpoint\machine_learning\one_step\tcb\ridge.pkl


In [37]:
evaluate(tcb_X_test, tcb_X_test_scaled, tcb_y_test, tcb_checkpoint_path)